# Deep Learning in Würzburg - Hands-On TensorFlow 2.0
## Dr. Tristan Behrens

Click this button if you want to enjoy Google Colab magic. Do not forget to change the runtime-type to use GPU!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AI-Guru/deep-learning-wuerzburg/blob/master/20200303%20FHWS%20Tensorflow%202/01%20MNIST.ipynb)



## Make sure that we have TensorFlow 2 installed.

In [ ]:
!pip install tensorflow==2.1.0

## Import all necessary modules  and check TensorFlow version.

In [ ]:
import tensorflow as tf
print(tf.__version__)

import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers

## Load and split MNIST-dataset.

- Train: For training the Neural Network.
- Validate: To see how good the Neural Network is after each epoch.
- Test: To see how good the Neural Network is after training.
- Link: [MNIST database](https://www.google.com/search?client=safari&rls=en&q=mnist&ie=UTF-8&oe=UTF-8).

In [ ]:
train_validation_split = tfds.Split.TRAIN.subsplit([8, 2])

(mnist_train, mnist_validate, mnist_test), info = tfds.load(
    name="mnist", 
    split=["train[:80%]", "train[80%:]", "test"],
    with_info=True,
    as_supervised=True
)

print("Train:   ", len(list(mnist_train)))
print("Validate:", len(list(mnist_validate)))
print("Test:    ", len(list(mnist_test)))

## Look at your \*beep\*-ing data!

This is supervised learning.

In [ ]:
for mnist_example in mnist_train.take(4):
    image, label = mnist_example

    plt.imshow(image.numpy()[:, :, 0], cmap=plt.get_cmap("gray"))
    plt.title("Label: {}".format(label.numpy()))
    plt.show()
    plt.close()

## Preparing the datasets with tf.data.

- Link: [tf.data: Build TensorFlow input pipelines](https://www.tensorflow.org/guide/data)

In [ ]:
def encode(image, label):
    image_encoded = tf.image.convert_image_dtype(image, dtype=tf.float32)
    label_encoded = tf.one_hot(label, depth=10)
    return image_encoded, label_encoded

mnist_train = mnist_train.map(lambda image, label: encode(image, label))
mnist_validate = mnist_validate.map(lambda image, label: encode(image, label))
mnist_test = mnist_test.map(lambda image, label: encode(image, label))

## Create a Deep Neural Network to solve our classification problem - Multi-Layer Perceptron.

In [ ]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(28, 28, 1)))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))
model.summary()

## Attach optimizer, loss, and metrics.

In [ ]:
model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

## How good is our ANN before training?

In [ ]:
mnist_example = list(mnist_test.take(1).as_numpy_iterator())[0]

image, label = mnist_example
prediction = model.predict(np.array([image]))
plt.plot(prediction[0])
plt.plot(label)
plt.show()
plt.close()

loss, acc = model.evaluate(mnist_test.batch(32), verbose=0)
print("Loss: {} Accuracy: {}".format(loss, acc))

## TODO Write.

In [ ]:
history = model.fit(
    mnist_train.batch(128),
    epochs=5,
    validation_data=mnist_validate.batch(128)
)

## TODO Write.

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.show()
plt.close()

plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.legend()
plt.show()
plt.close()

## How good is our ANN after training?

In [ ]:
mnist_example = list(mnist_test.take(1).as_numpy_iterator())[0]

image, label = mnist_example
prediction = model.predict(np.array([image]))
plt.plot(prediction[0])
plt.plot(label)
plt.show()
plt.close()

loss, acc = model.evaluate(mnist_test.batch(32), verbose=0)
print("Loss: {} Accuracy: {}".format(loss, acc))

## Save the model.

In [ ]:
model.save("model.h5")